In [1]:
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, MaxPooling2D)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
IMG_HEIGHT = 256
IMG_WIDTH  = 256
CHANNELS   = 3
batch_size = 16

In [3]:
train_path = "Data_rotation//Images/"
test_path  = "Data_rotation//Evaluation/"
class_names = ['degree_0', 'degree_90', 'degree_180', 'degree_270']

In [4]:

## Generators ##
 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.15
)

# Training
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)

# Validation
validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation'
)

# Evaluation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size
)

Found 3448 images belonging to 4 classes.
Found 604 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [5]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))

model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))

model.add(Conv2D(64, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))

model.add(Conv2D(128, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 254, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 125, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 125, 125, 32)      0

In [6]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:

early_stop = EarlyStopping(monitor='val_loss', patience=5)
lr_redux = ReduceLROnPlateau(monitor='val_loss',
                            patience = 3, verbose = 1,
                            factor = 0.1, min_lr = 0.000001)
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')


log_dir = f"/home/daisy/mydrive/logs/{int(time.time())}"

tensorboard = TensorBoard(log_dir=log_dir)
callbacks = [early_stop, lr_redux, tensorboard]

In [8]:
# Fit model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator,
    callbacks=callbacks
)
model.save('rotation_main.h5')


  1/216 [..............................] - ETA: 0s - loss: 1.7623 - accuracy: 0.3750WARNING:tensorflow:From /home/daisy/mydrive/datascience/environments/main_env/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
216/216 [==============================] - 124s 576ms/step - loss: 2.0200 - accuracy: 0.3744 - val_loss: 1.3867 - val_accuracy: 0.2500


In [9]:
# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)

print("Test accuracy: {}" .format(test_acc))
print("Test loss: {}" .format(test_loss))

7/7 [==============================] - 3s 391ms/step - loss: 1.3660 - accuracy: 0.3100
Test accuracy: 0.3100000023841858
Test loss: 1.3659745454788208
